<a href="https://colab.research.google.com/github/michal-g/Notebooks-to-Packages/blob/main/1-Notebooks/predicting-ufo-sightings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start by acquiring the data on UFO sightings. Fortunately, github user Link Wentz has been nice enough to download
it from the website and parse it into a reasonable format, which we make available here.

We notice however that there are irregularities in some of the entries, which we would like to clean up!

In [ ]:
import pandas as pd

sightings = pd.read_csv('../nuforc_events_complete.csv',
                        usecols=['event_time', 'city', 'state',
                                 'shape', 'duration', 'summary'])

print(sightings.head())
print('----------')
print("Number of duplicate rows:")
print(sightings.duplicated().sum())
print('----------')
print("# of rows by state:")
print(sightings.state.value_counts())


In [ ]:
# we will be very careful about filtering sightings that can be mapped to states
valid_states = {
    'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI',
    'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN',
    'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH',
    'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
    'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'
    }
sightings = sightings.loc[sightings.state.isin(valid_states), :]

# parse the date information into a more useful format
sightings['event_time'] = pd.to_datetime(sightings.event_time,
                                         format="%Y-%m-%dT%H:%M:%SZ", errors='coerce')
print(f"{sightings['event_time'].isna().sum()} rows with missing times")

sightings = sightings.loc[~sightings['event_time'].isna(), :]
print(sightings.head())
print(sightings.tail())
print("---------")
print(f"{sightings.shape[0]} total sightings")


Let's examine the total number of sightings from across the year for each state. We see that California
is a very popular destination!

In [ ]:
import plotly.express as px


state_totals = sightings.groupby('state').size()

fig = px.choropleth(locations=[str(x) for x in state_totals.index],
                    scope="usa", locationmode="USA-states",
                    color=state_totals.values,
                    range_color=[0, state_totals.max()],
                    color_continuous_scale=['white', 'red'])

fig.update_layout(coloraxis_colorbar=dict(title="Sightings"))

fig.show()


To better understand how sightings vary across the year, we'll animate the weekly numbers of sightings for each state.
Note the use of multi-dimensional `groupby` to create a table with sightings broken down by time and place.

In [ ]:
import imageio
from IPython.display import Image
from pathlib import Path


# create a Year x State table containing total yearly sightings for each state;
# note that we have to take into account "missing" years that did not have any
# sightings in any states
state_yearlies = sightings.groupby(
    [sightings.event_time.dt.year, 'state']).size().unstack().fillna(0)

# create a list of individual files and a place to save them
plt_files = list()
!mkdir -p map-plots

# create a map of sightings by state for each week
for year, year_counts in state_yearlies.iterrows():
    state_locs = [str(x) for x in year_counts.index.get_level_values('state')]

    fig = px.choropleth(locations=state_locs, locationmode="USA-states",
                        title=year, scope='usa',
                        color=year_counts.values, range_color=[0, 10],
                        color_continuous_scale=['white', 'black'])
    fig.update_layout(coloraxis_colorbar=dict(title="Sightings"))

    # save the map to file and keep track of the file name
    plt_file = Path("map-plots", f"counts_{year}.png")
    fig.write_image(plt_file, format='png')
    plt_files += [imageio.v2.imread(plt_file)]

# create an animation using the individual weekly maps
imageio.mimsave(Path("map-plots", "counts.gif"), plt_files, duration=0.03)
Image(filename=str(Path("map-plots", "counts.gif")))


A time series regression model can predict the number of sightings that will take place on each of a series of weeks.
We set up a cross-validation process in which the model makes predictions across chunks of our year range by training
on preceding chunks. The quality of the predictions is judged using root-mean-squared-error, which is useful as it
places the error in the same scale as the original data.

To improve our model's performance, we can try to swap out or rearrange elements of the prediction pipeline, as well
as tune the hyper-parameters of the input transformers and prediction algorithms. This is left as an exercise
to the reader!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (14, 9)
import numpy as np

from skits.preprocessing import ReversibleImputer
from skits.pipeline import ForecasterPipeline
from skits.feature_extraction import (AutoregressiveTransformer,
                                      SeasonalTransformer)

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit


pipeline = ForecasterPipeline([
    ('pre_scaler', StandardScaler()),
    ('features', FeatureUnion([
      ('ar_features', AutoregressiveTransformer(num_lags=52)),
      ('seasonal_features', SeasonalTransformer(seasonal_period=52)),
      ])),
    ('post_feature_imputer', ReversibleImputer()),
    ('post_feature_scaler', StandardScaler()),
    ('regressor', LinearRegression())
    ])

# assets and specially formatted objects used by the prediction pipeline
# scikit-learn wants Xs to be 2-dimensional and ys to be 1-dimensional
tscv = TimeSeriesSplit(n_splits=4)
cali_yearlies = state_yearlies.CA.reindex(
    range(state_yearlies.index[0], state_yearlies.index[-1] + 1),
    fill_value=0.
    )

print(cali_yearlies.tail())

cali_dates = cali_yearlies.index.values.reshape(-1, 1)
cali_values = cali_yearlies.values

real_values = list()
pred_values = list()

# for each cross-validation fold, use the training samples in the fold to
# train the pipeline and the remaining samples to test it
for train_index, test_index in tscv.split(cali_yearlies):
    pipeline.fit(cali_dates[train_index], cali_values[train_index])
    preds = pipeline.predict(cali_dates[test_index], to_scale=True)

    # we'll keep track of the actual sightings and the predicted sightings from
    # each c-v fold for future reference
    real_values += cali_values[test_index].flatten().tolist()
    pred_values += preds.flatten().tolist()

    plt.plot(cali_dates[test_index], cali_values[test_index], color='black')
    plt.plot(cali_dates[test_index], preds, color='red')

# measure the quality of the predictions using root-mean-squared-error
rmse_val = ((np.array(real_values) - np.array(pred_values)) ** 2).mean() ** 0.5
print(f"RMSE: {format(rmse_val, '.3f')}")
